# Air Quality in Lagos

In [1]:
# Import libraries here
import time
import warnings

from pprint import PrettyPrinter
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import seaborn as sns
from pymongo import MongoClient
from sklearn.metrics import mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.ar_model import AutoReg

warnings.filterwarnings("ignore")

## Prepare Data

### Connect

In [2]:
pp = PrettyPrinter(indent=2)


In [4]:
client = MongoClient(host='localhost', port=27017)
db = client['air-quality']

In [5]:
for c in db.list_collections():
    print(c['name'])


ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 6428acf5515798f0cefc8150, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 61] Connection refused')>]>